In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint

C:\Users\Hao\AppData\Local\conda\conda\envs\MLOntology\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
#read class label file
#create mapping from id to labels





In [2]:

conceptMappingDict={}

In [3]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            splitted = line.decode('iso-8859-1').split("\t", 1)
            
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                conceptMappingDict[i]= int(splitted[0])
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [ ]:
#gensim.utils.simple_preprocess??
gensim.models.doc2vec.TaggedDocument??

In [4]:
train_file = "ontClassTopology.txt"

train_corpus = list(read_corpus(train_file))

In [5]:
print(len(train_corpus))
train_corpus[:2]

336886


[TaggedDocument(words=['snomed', 'ct', 'concept', 'physical', 'object', 'device', 'clinical', 'equipment', 'and', 'or', 'device', 'biomedical', 'equipment', 'laboratory', 'instruments', 'and', 'equipment', 'culture', 'plate', 'air', 'sampling', 'culture', 'plate', 'air', 'sampling', 'culture', 'plate', 'air', 'sampling', 'culture', 'plate'], tags=[0]),
 TaggedDocument(words=['snomed', 'ct', 'concept', 'clinical', 'finding', 'evaluation', 'finding', 'morphologic', 'finding', 'size', 'finding', 'increased', 'diameter', 'increased', 'diameter', 'increased', 'diameter'], tags=[1])]

In [6]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)

In [7]:
model.build_vocab(train_corpus)

In [ ]:
len(model.wv.vocab)

In [8]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

KeyboardInterrupt: 

In [17]:
# inferred_vector = model.infer_vector(['clinical', 'finding', 'evaluation', 'prevent', 'sampling', 'foot'])
inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])


sims = model.docvecs.most_similar([inferred_vector], topn=10)


for sim in sims:
    print(train_corpus[sim[0]].words, "score: ", sim[1])
    
for sim in sims:
    print(conceptMappingDict[sim[0]], "score: ", sim[1])
  

['snomed', 'ct', 'concept', 'clinical', 'finding', 'disease', 'complication', 'complication', 'of', 'procedure', 'complication', 'associated', 'with', 'device', 'complication', 'of', 'implant', 'complication', 'of', 'internal', 'prosthetic', 'device', 'complication', 'of', 'internal', 'prosthetic', 'device', 'complication', 'of', 'internal', 'prosthetic', 'device', 'disorder', 'of', 'bile', 'duct', 'stent', 'complication', 'associated', 'with', 'saline', 'breast', 'implant', 'disorder', 'of', 'bile', 'duct', 'prosthesis', 'cardiac', 'insufficiency', 'due', 'to', 'prosthesis', 'internal', 'prosthetic', 'device', 'causing', 'pain', 'disorder', 'of', 'implanted', 'penile', 'prosthesis', 'disorder', 'of', 'testicular', 'prosthesis', 'disorder', 'of', 'prosthetic', 'joint', 'internal', 'prosthetic', 'device', 'causing', 'stenosis', 'mechanical', 'hemolysis', 'due', 'to', 'cardiovascular', 'prosthetic', 'patch', 'internal', 'prosthetic', 'device', 'causing', 'embolism', 'internal', 'prosthet

In [9]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

In [10]:
new_model = gensim.models.Doc2Vec.load(temp_path)

In [12]:
cores = multiprocessing.cpu_count()

models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, size=200, window=8, min_count=1, iter=10, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=200, window=8, min_count=1, iter =10, workers=cores),
]


In [13]:
models[0].build_vocab(train_corpus)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

Doc2Vec(dbow+w,d200,n5,w8,s0.001,t4)
Doc2Vec(dm/m,d200,n5,w8,s0.001,t4)


In [15]:
for model in models:
    %%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
for model in models:
    print(str(model))
    pprint(model.docvecs.most_similar(positive=["Machine learning"], topn=20))

In [ ]:
for model in models:
    print(str(model))
    vec = [model.docvecs["Lady Gaga"] - model["american"] + model["japanese"]]
    pprint([m for m in model.docvecs.most_similar(vec, topn=11) if m[0] != "Lady Gaga"])

In [ ]:
print("Total: ",len(train_corpus))


ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    print("Processing: ", doc_id)
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [ ]:
collections.Counter(ranks) 